<a href="https://colab.research.google.com/github/SangLee5661/New-Folder2/blob/main/%EC%8B%A0%EC%9A%A9%EC%9C%84%ED%97%98%EB%8F%84_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/card_train.csv'
test_path = '/content/drive/MyDrive/card_test.csv'
train_df = pd.read_csv(train_path)

Mounted at /content/drive


In [3]:
print("데이터 형태:", train_df.shape)
print(train_df.head())

print("\n전체 컬럼 목록:")
for i, col in enumerate(train_df.columns):
    print(f"{i+1:2d}. {col}")

데이터 형태: (70560, 738)
   Unnamed: 0.1  대표결제일 대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법  \
0         61865      1     자동이체           주거지           우편      우편   
1          8547     13     자동이체           미확인          이메일     이메일   
2         43497     25     자동이체           주거지           우편      우편   
3         40785     25     자동이체           주거지           우편      우편   
4         18999     25     자동이체           주거지           우편      우편   

   청구서발송여부_B0  청구서발송여부_R3M  청구서발송여부_R6M  청구금액_B0  ...  컨택건수_CA_청구서_R6M  \
0           1            1            1      466  ...                0   
1           1            1            1     2417  ...                0   
2           0            0            0        0  ...                2   
3           1            1            1     5931  ...                0   
4           1            1            1      684  ...                2   

   컨택건수_이용유도_청구서_R6M  컨택건수_이용유도_인터넷_R6M  컨택건수_이용유도_당사앱_R6M  컨택건수_채권_B0M  \
0                  5                  

In [4]:
X = train_df.drop(columns=['ID', 'Unnamed: 0.1', 'Segment.1', 'Segment'])
Y = train_df['Segment']

print("\n타겟 분포:")
print(Y.value_counts())


타겟 분포:
Segment
E    56505
D    10270
C     3753
A       28
B        4
Name: count, dtype: int64


In [5]:
# 다양한 기간별 피처 추출 전략 (개선됨)
def extract_comprehensive_features(X):
    """포괄적인 피처 추출 전략"""
    print("\n=== 포괄적인 피처 추출 전략 ===")

    # 1. 기간별 피처 그룹핑
    feature_groups = {
        'R3M': [], 'R6M': [], 'R12M': [], 'L3M': [], 'L6M': [], 'L12M': [],
        'AVG': [], 'MAX': [], 'MIN': [], 'STATIC': []
    }

    # 2. 데이터 타입별 피처 그룹핑
    data_type_groups = {
        '신용정보': [], '승인매출': [], '청구잔액': [], '채널이용': [],
        '마케팅': [], '성과지표': [], '기타': []
    }

    # 컬럼명 분석 및 그룹핑
    for col in X.columns:
        col_lower = col.lower()

        # 기간별 분류
        if any(p in col for p in ['R3M', 'r3m', '_3m', '3개월']):
            feature_groups['R3M'].append(col)
        elif any(p in col for p in ['R6M', 'r6m', '_6m', '6개월']):
            feature_groups['R6M'].append(col)
        elif any(p in col for p in ['R12M', 'r12m', '_12m', '12개월']):
            feature_groups['R12M'].append(col)
        elif any(p in col for p in ['L3M', 'l3m']):
            feature_groups['L3M'].append(col)
        elif any(p in col for p in ['L6M', 'l6m']):
            feature_groups['L6M'].append(col)
        elif any(p in col for p in ['L12M', 'l12m']):
            feature_groups['L12M'].append(col)
        elif any(k in col_lower for k in ['avg', '평균', 'mean']):
            feature_groups['AVG'].append(col)
        elif any(k in col_lower for k in ['max', '최대', 'maximum']):
            feature_groups['MAX'].append(col)
        elif any(k in col_lower for k in ['min', '최소', 'minimum']):
            feature_groups['MIN'].append(col)
        else:
            feature_groups['STATIC'].append(col)

        # 데이터 타입별 분류
        if any(k in col_lower for k in ['신용', 'credit', '한도', '등급', 'grade', 'limit']):
            data_type_groups['신용정보'].append(col)
        elif any(k in col_lower for k in ['승인', '매출', '사용', '결제', 'payment', 'purchase', 'amount']):
            data_type_groups['승인매출'].append(col)
        elif any(k in col_lower for k in ['청구', '잔액', '연체', 'balance', 'bill', 'due']):
            data_type_groups['청구잔액'].append(col)
        elif any(k in col_lower for k in ['채널', 'channel', '온라인', '오프라인', 'pos', 'web', 'mobile']):
            data_type_groups['채널이용'].append(col)
        elif any(k in col_lower for k in ['마케팅', 'marketing', '캠페인', 'campaign', '프로모션', 'promotion']):
            data_type_groups['마케팅'].append(col)
        elif any(k in col_lower for k in ['포인트', 'point', '리워드', 'reward', '혜택', 'benefit']):
            data_type_groups['성과지표'].append(col)
        else:
            data_type_groups['기타'].append(col)

    # 결과 출력
    print("\n기간별 피처 분포:")
    for period, features in feature_groups.items():
        if features:
            print(f"{period}: {len(features)}개")
            if len(features) <= 5:
                print(f"  → {features}")
            else:
                print(f"  → {features[:3]} ... (총 {len(features)}개)")

    print("\n데이터 타입별 피처 분포:")
    for data_type, features in data_type_groups.items():
        if features:
            print(f"{data_type}: {len(features)}개")
            if len(features) <= 3:
                print(f"  → {features}")

    return feature_groups, data_type_groups

In [6]:
# 전략적 피처 선택
def strategic_feature_selection(X, feature_groups, data_type_groups, strategy='comprehensive'):
    """전략적 피처 선택"""
    selected_features = []

    if strategy == 'comprehensive':
        print("\n=== 포괄적 전략: 모든 기간과 데이터 타입 활용 ===")
        for period_features in feature_groups.values():
            selected_features.extend(period_features)
        selected_features = list(set(selected_features))
        if len(selected_features) < 10:
            selected_features = list(X.columns)

    elif strategy == 'trend_focused':
        print("\n=== 트렌드 중심 전략: 시계열 변화 패턴 분석 ===")
        priority_periods = ['R3M', 'R6M', 'R12M', 'STATIC']
        for period in priority_periods:
            selected_features.extend(feature_groups[period])
        if len(selected_features) < 10:
            for period in ['L3M', 'L6M', 'AVG']:
                selected_features.extend(feature_groups[period])

    elif strategy == 'credit_focused':
        print("\n=== 신용 중심 전략: 신용도 관련 피처 집중 ===")
        priority_types = ['신용정보', '승인매출', '청구잔액']
        for data_type in priority_types:
            selected_features.extend(data_type_groups[data_type])
        selected_features.extend(feature_groups['R3M'])
        if len(selected_features) < 10:
            for data_type in ['채널이용', '기타']:
                selected_features.extend(data_type_groups[data_type])

    elif strategy == 'business_focused':
        print("\n=== 비즈니스 중심 전략: 수익성과 활용도 중심 ===")
        priority_types = ['승인매출', '채널이용', '성과지표']
        for data_type in priority_types:
            selected_features.extend(data_type_groups[data_type])
        selected_features.extend(feature_groups['R3M'])
        selected_features.extend(feature_groups['R6M'])
        if len(selected_features) < 10:
            selected_features.extend(data_type_groups['마케팅'])
            selected_features.extend(data_type_groups['기타'])

    selected_features = list(set(selected_features))
    selected_features = [f for f in selected_features if f in X.columns]

    if len(selected_features) < 5:
        print("⚠️ 선택된 피처가 너무 적습니다. 모든 피처를 사용합니다.")
        selected_features = list(X.columns)

    print(f"선택된 피처 수: {len(selected_features)}")
    return selected_features

In [7]:
def safe_smote_fit(X_train, y_train, min_samples_per_class=5):
    """SMOTE를 안전하게 적용하는 함수"""
    class_counts = pd.Series(y_train).value_counts()
    min_count = class_counts.min()

    if min_count < min_samples_per_class:
        print(f"⚠️ 일부 클래스의 샘플이 너무 적음 (최소: {min_count}). SMOTE 적용 불가.")
        return X_train, y_train

    valid_classes = class_counts[class_counts >= min_samples_per_class].index
    if len(valid_classes) < len(class_counts):
        print(f"극소수 클래스 제외: {set(class_counts.index) - set(valid_classes)}")
        mask = pd.Series(y_train).isin(valid_classes)
        X_train = X_train[mask]
        y_train = pd.Series(y_train)[mask]

    min_count = pd.Series(y_train).value_counts().min()
    k_neighbors = min(5, min_count - 1)

    try:
        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_balanced, y_balanced = smote.fit_resample(X_train, y_train)
        print(f"✅ SMOTE 적용 완료 (k_neighbors={k_neighbors})")
        return X_balanced, y_balanced
    except Exception as e:
        print(f"❌ SMOTE 적용 실패: {str(e)}")
        return X_train, y_train

In [8]:
# 피처 추출 및 선택 실행
feature_groups, data_type_groups = extract_comprehensive_features(X)


=== 포괄적인 피처 추출 전략 ===

기간별 피처 분포:
R3M: 92개
  → ['청구서발송여부_R3M', '청구금액_R3M', '포인트_마일리지_건별_R3M'] ... (총 92개)
R6M: 144개
  → ['청구서발송여부_R6M', '청구금액_R6M', '상환개월수_결제일_R6M'] ... (총 144개)
R12M: 75개
  → ['포인트_적립포인트_R12M', '포인트_이용포인트_R12M', '마일_적립포인트_R12M'] ... (총 75개)
MIN: 1개
  → ['RV최소결제비율']
STATIC: 422개
  → ['대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드'] ... (총 422개)

데이터 타입별 피처 분포:
신용정보: 73개
승인매출: 42개
청구잔액: 79개
채널이용: 29개
마케팅: 3개
  → ['마케팅동의여부', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M']
성과지표: 25개
기타: 483개


In [9]:
# 여러 전략으로 피처 선택 테스트
strategies = ['comprehensive', 'trend_focused', 'credit_focused', 'business_focused']
strategy_results = {}

for strategy in strategies:
    print(f"\n{'='*50}")
    selected_features = strategic_feature_selection(X, feature_groups, data_type_groups, strategy)

    if len(selected_features) == 0:
        print(f"⚠️ {strategy} 전략으로 선택된 피처가 없습니다.")
        continue

    x_selected = X[selected_features].copy()

    # 범주형 변수 처리
    categorical_cols = [col for col in x_selected.columns if x_selected[col].dtype == 'object']
    if categorical_cols:
        print(f"범주형 변수 ({len(categorical_cols)}개): {categorical_cols[:3]}...")
        for col in categorical_cols:
            le = LabelEncoder()
            x_selected[col] = le.fit_transform(x_selected[col].astype(str))

    # 결측치 처리
    x_selected = x_selected.fillna(x_selected.median())

    # 무한값 처리
    x_selected = x_selected.replace([np.inf, -np.inf], np.nan)
    x_selected = x_selected.fillna(x_selected.median())

    # 신용위험도 피처 생성
    print(f"\n{strategy} 전략 신용위험도 피처 생성...")

    usage_patterns = ['사용금액', '사용액', 'usage', 'amount']
    limit_patterns = ['한도금액', '한도액', 'limit']

    usage_cols = []
    limit_cols = []

    for pattern in usage_patterns:
        usage_cols.extend([col for col in x_selected.columns if pattern in col])
    for pattern in limit_patterns:
        limit_cols.extend([col for col in x_selected.columns if pattern in col])

    # 사용률 피처 생성
    ratio_created = 0
    for usage_col in usage_cols:
        for limit_col in limit_cols:
            if any(c in usage_col and c in limit_col for c in ['R3M', 'R6M', 'R12M', '신용', '체크']):
                ratio_name = f"사용률_{usage_col.split('_')[-1] if '_' in usage_col else 'general'}"
                safe_limit = x_selected[limit_col].replace(0, 1)
                x_selected[ratio_name] = x_selected[usage_col] / safe_limit
                ratio_created += 1
                if ratio_created <= 3:
                    print(f"  → {ratio_name} 생성")

    # 할인 의존도
    discount_cols = [col for col in x_selected.columns if '할인' in col or 'discount' in col]
    for discount_col in discount_cols[:3]:
        usage_col = None
        for usage in usage_cols:
            if any(c in discount_col and c in usage for c in ['R3M', 'R6M', 'R12M']):
                usage_col = usage
                break
        if usage_col:
            depend_name = f"할인의존도_{discount_col.split('_')[-1] if '_' in discount_col else 'general'}"
            safe_usage = x_selected[usage_col].replace(0, 1)
            x_selected[depend_name] = x_selected[discount_col] / safe_usage
            print(f"  → {depend_name} 생성")

    # 거래 빈도
    count_patterns = ['이용건수', '거래건수', '건수', 'count']
    count_cols = []
    for pattern in count_patterns:
        count_cols.extend([col for col in x_selected.columns if pattern in col])

    for count_col in count_cols[:3]:
        if 'R3M' in count_col or '3개월' in count_col:
            freq_name = f"월평균빈도_{count_col.split('_')[-1] if '_' in count_col else 'general'}"
            x_selected[freq_name] = x_selected[count_col] / 3
            print(f"  → {freq_name} 생성")

    print(f"최종 피처 수: {x_selected.shape[1]}")

    # 간단한 성능 테스트
    if x_selected.shape[1] > 0:
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                x_selected, Y, test_size=0.2, random_state=42, stratify=Y
            )

            X_train_balanced, y_train_balanced = safe_smote_fit(X_train, y_train)

            rf_test = RandomForestClassifier(n_estimators=50, random_state=42, class_weight='balanced')
            rf_test.fit(X_train_balanced, y_train_balanced)

            y_pred = rf_test.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)

            print(f"🎯 {strategy} 전략 기본 성능: {accuracy:.3f}")

            strategy_results[strategy] = {
                'features': selected_features,
                'accuracy': accuracy,
                'feature_count': x_selected.shape[1],
                'data': x_selected
            }

        except Exception as e:
            print(f"❌ {strategy} 전략 성능 테스트 실패: {str(e)}")
            strategy_results[strategy] = {
                'features': selected_features,
                'accuracy': 0,
                'feature_count': x_selected.shape[1],
                'data': x_selected
            }



=== 포괄적 전략: 모든 기간과 데이터 타입 활용 ===
선택된 피처 수: 734
범주형 변수 (45개): ['대표청구지고객주소구분코드', 'RV전환가능여부', '_2순위납부업종']...

comprehensive 전략 신용위험도 피처 생성...
  → 월평균빈도_R3M 생성
최종 피처 수: 735
⚠️ 일부 클래스의 샘플이 너무 적음 (최소: 3). SMOTE 적용 불가.
🎯 comprehensive 전략 기본 성능: 0.873


=== 트렌드 중심 전략: 시계열 변화 패턴 분석 ===
선택된 피처 수: 733
범주형 변수 (45개): ['대표청구지고객주소구분코드', 'RV전환가능여부', '_2순위납부업종']...

trend_focused 전략 신용위험도 피처 생성...
  → 월평균빈도_R3M 생성
최종 피처 수: 734
⚠️ 일부 클래스의 샘플이 너무 적음 (최소: 3). SMOTE 적용 불가.
🎯 trend_focused 전략 기본 성능: 0.874


=== 신용 중심 전략: 신용도 관련 피처 집중 ===
선택된 피처 수: 253
범주형 변수 (10개): ['대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']...

credit_focused 전략 신용위험도 피처 생성...
  → 월평균빈도_R3M 생성
  → 월평균빈도_R3M 생성
최종 피처 수: 254
⚠️ 일부 클래스의 샘플이 너무 적음 (최소: 3). SMOTE 적용 불가.
🎯 credit_focused 전략 기본 성능: 0.870


=== 비즈니스 중심 전략: 수익성과 활용도 중심 ===
선택된 피처 수: 272
범주형 변수 (7개): ['인입횟수_ARS_R6M', '방문일수_PC_R6M', '이용메뉴건수_ARS_R6M']...

business_focused 전략 신용위험도 피처 생성...
  → 월평균빈도_R3M 생성
  → 월평균빈도_R3M 생성
최종 피처 수: 273
⚠️ 일부 클래스의 샘플이 너무 적음 (최소: 3). SMOTE 적용 불가.
🎯 b

In [10]:
# 최적 전략 선택
if strategy_results:
    print(f"\n{'='*50}")
    print("전략별 성능 비교:")
    for strategy, result in strategy_results.items():
        print(f"{strategy:15s}: 정확도 {result['accuracy']:.3f}, 피처 수 {result['feature_count']:3d}")

    best_strategy = max(strategy_results.keys(), key=lambda x: strategy_results[x]['accuracy'])
    print(f"\n🏆 최적 전략: {best_strategy} (정확도: {strategy_results[best_strategy]['accuracy']:.3f})")

    x_train = strategy_results[best_strategy]['data']
    selected_features = strategy_results[best_strategy]['features']

    print(f"\n최종 선택된 피처들 (상위 20개):")
    for i, feature in enumerate(selected_features[:20]):
        print(f"{i+1:2d}. {feature}")
    if len(selected_features) > 20:
        print(f"    ... 외 {len(selected_features)-20}개")

    # 신용위험도 가설 검증
    print(f"\n{'='*50}")
    print("=== 신용위험도 가설 검증 ===")
    print("가설: E등급이 가장 높은 신용위험도를 보인다")

    risk_score = pd.Series(0.0, index=x_train.index)
    risk_components = []

    usage_ratio_cols = [col for col in x_train.columns if '사용률' in col]
    for col in usage_ratio_cols:
        if x_train[col].std() > 0:
            standardized = (x_train[col] - x_train[col].mean()) / x_train[col].std()
            risk_score += standardized
            risk_components.append(f"사용률({col})")

    discount_ratio_cols = [col for col in x_train.columns if '할인의존도' in col]
    for col in discount_ratio_cols:
        if x_train[col].std() > 0:
            standardized = (x_train[col] - x_train[col].mean()) / x_train[col].std()
            risk_score += standardized
            risk_components.append(f"할인의존도({col})")

    freq_cols = [col for col in x_train.columns if '빈도' in col or '건수' in col]
    for col in freq_cols:
        if x_train[col].std() > 0:
            standardized = (x_train[col] - x_train[col].mean()) / x_train[col].std()
            risk_score += -standardized
            risk_components.append(f"거래빈도({col})")

    print(f"위험점수 구성요소 ({len(risk_components)}개): {risk_components[:5]}...")

    if len(risk_components) > 0:
        segment_risk = pd.DataFrame({
            'Segment': Y,
            'Risk_Score': risk_score
        }).groupby('Segment')['Risk_Score'].agg(['mean', 'std', 'count']).sort_values('mean', ascending=False)

        print("\n등급별 평균 위험점수 (높을수록 위험):")
        print(segment_risk)

        if len(segment_risk) > 0:
            highest_risk_segment = segment_risk.index[0]
            print(f"\n가장 높은 위험점수 등급: {highest_risk_segment}")

            if 'E' in segment_risk.index:
                e_risk = segment_risk.loc['E', 'mean']
                print(f"E등급 위험점수: {e_risk:.3f}")
                if highest_risk_segment == 'E':
                    print("✅ 가설 성립: E등급이 가장 높은 신용위험도를 보임")
                else:
                    print("❌ 가설 불성립: E등급이 가장 높은 신용위험도를 보이지 않음")
                    print(f"실제로는 {highest_risk_segment}등급이 가장 위험함")
            else:
                print("⚠️ E등급이 데이터에 없습니다.")
    else:
        print("⚠️ E등급 데이터가 없어 1단계 분류를 수행할 수 없습니다.")

else:
    print("⚠️ 모든 전략에서 성능 테스트에 실패했습니다.")
    print("기본 전략으로 단일 모델 학습을 시도합니다.")

    x_all = X.copy()

    categorical_cols = [col for col in x_all.columns if x_all[col].dtype == 'object']
    if categorical_cols:
        print(f"범주형 변수 ({len(categorical_cols)}개) 처리 중...")
        for col in categorical_cols:
            le = LabelEncoder()
            x_all[col] = le.fit_transform(x_all[col].astype(str))

    x_all = x_all.fillna(x_all.median())
    x_all = x_all.replace([np.inf, -np.inf], np.nan)
    x_all = x_all.fillna(x_all.median())

    print(f"기본 전략 피처 수: {x_all.shape[1]}")

    try:
        X_train, X_test, y_train, y_test = train_test_split(
            x_all, Y, test_size=0.2, random_state=42, stratify=Y
        )

        X_train_balanced, y_train_balanced = safe_smote_fit(X_train, y_train)

        rf_basic = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        rf_basic.fit(X_train_balanced, y_train_balanced)

        y_pred_basic = rf_basic.predict(X_test)
        accuracy_basic = accuracy_score(y_test, y_pred_basic)

        print(f"🎯 기본 전략 성능: {accuracy_basic:.3f}")

        print("\n등급별 분류 성능:")
        print(classification_report(y_test, y_pred_basic))

    except Exception as e:
        print(f"❌ 기본 전략도 실패: {str(e)}")

print(f"\n{'='*50}")
print("분석 완료!")
if 'best_strategy' in locals():
    print(f"최종 사용된 전략: {best_strategy}")
    print(f"최종 피처 수: {len(selected_features)}")
else:
    print("기본 전략 사용")
print("다양한 기간과 데이터 타입을 활용한 포괄적 분석이 수행되었습니다.")


전략별 성능 비교:
comprehensive  : 정확도 0.873, 피처 수 735
trend_focused  : 정확도 0.874, 피처 수 734
credit_focused : 정확도 0.870, 피처 수 254
business_focused: 정확도 0.857, 피처 수 273

🏆 최적 전략: trend_focused (정확도: 0.874)

최종 선택된 피처들 (상위 20개):
 1. 이용개월수_신판_R3M
 2. 승인거절건수_R3M
 3. 컨택건수_리볼빙_LMS_B0M
 4. 한도심사요청후경과월
 5. 이용금액_카드론_R3M
 6. 이용금액_A페이_R3M
 7. IB문의건수_결제_R6M
 8. 이용개월수_신용_R6M
 9. 일시상환론한도금액
10. 대표청구지고객주소구분코드
11. 월상환론한도금액
12. 동의여부_한도증액안내
13. 이용금액_C페이_R3M
14. 이용건수_체크_B0M
15. 마일_적립포인트_R3M
16. 이용건수_일시불_R6M
17. 증감율_이용금액_신용_전월
18. 이용후경과월_할부_유이자
19. 월중평잔_카드론
20. 이용금액_CA_R12M
    ... 외 713개

=== 신용위험도 가설 검증 ===
가설: E등급이 가장 높은 신용위험도를 보인다
위험점수 구성요소 (206개): ['거래빈도(승인거절건수_R3M)', '거래빈도(컨택건수_리볼빙_LMS_B0M)', '거래빈도(IB문의건수_결제_R6M)', '거래빈도(이용건수_체크_B0M)', '거래빈도(이용건수_일시불_R6M)']...

등급별 평균 위험점수 (높을수록 위험):
               mean         std  count
Segment                               
E         11.055027   38.622740  56505
D        -38.792811   55.133700  10270
C        -59.483871   62.836863   3753
A        -81.790339   52.065485  

In [11]:
# 피처 중요도 분석
if 'rf_basic' in locals():
    print(f"\n{'='*50}")
    print("=== 피처 중요도 분석 ===")

    feature_importance = pd.DataFrame({
        'feature': x_all.columns,
        'importance': rf_basic.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\n상위 20개 중요 피처:")
    for i, (_, row) in enumerate(feature_importance.head(20).iterrows()):
        print(f"{i+1:2d}. {row['feature']:30s} {row['importance']:.4f}")

    print("\n피처 타입별 중요도:")
    type_importance = {}

    for _, row in feature_importance.iterrows():
        feature_name = row['feature'].lower()
        importance = row['importance']

        if any(p in feature_name for p in ['r3m', '3개월']):
            type_importance['최근3개월'] = type_importance.get('최근3개월', 0) + importance
        elif any(p in feature_name for p in ['r6m', '6개월']):
            type_importance['최근6개월'] = type_importance.get('최근6개월', 0) + importance
        elif any(p in feature_name for p in ['r12m', '12개월']):
            type_importance['최근12개월'] = type_importance.get('최근12개월', 0) + importance
        elif any(p in feature_name for p in ['avg', '평균']):
            type_importance['평균값'] = type_importance.get('평균값', 0) + importance
        else:
            type_importance['기타'] = type_importance.get('기타', 0) + importance

    for type_name, total_importance in sorted(type_importance.items(), key=lambda x: x[1], reverse=True):
        print(f"{type_name:10s}: {total_importance:.4f}")

In [12]:
# 비즈니스 인사이트 제공
print(f"\n{'='*50}")
print("=== 비즈니스 인사이트 ===")

print("\n1. 데이터 품질 분석:")
print(f"   - 총 피처 수: {X.shape[1]}")
print(f"   - 총 샘플 수: {X.shape[0]}")
print(f"   - 등급 분포: {dict(Y.value_counts())}")

missing_ratio = X.isnull().sum().sum() / (X.shape[0] * X.shape[1])
print(f"   - 결측치 비율: {missing_ratio:.3f}")

print("\n2. 등급별 특성:")
for grade in sorted(Y.unique()):
    grade_count = (Y == grade).sum()
    grade_ratio = grade_count / len(Y)
    print(f"   - {grade}등급: {grade_count}명 ({grade_ratio:.1%})")

print("\n3. 모델링 권장사항:")
print("   - 클래스 불균형 문제가 있으므로 SMOTE나 class_weight 조정 필요")
print("   - 2단계 분류 시스템으로 E등급을 먼저 식별하는 전략 유효")
print("   - 최근 3-6개월 데이터가 가장 중요한 예측 변수로 판단됨")
print("   - 신용카드 사용률, 거래 빈도 등이 핵심 지표")

print("\n4. 추가 개선 방안:")
print("   - 피처 엔지니어링을 통한 비율/변화율 변수 생성")
print("   - 시간 기반 피처 중요도를 고려한 가중치 적용")
print("   - 앙상블 모델 적용으로 성능 향상")
print("   - 교차 검증을 통한 모델 안정성 검증")


=== 비즈니스 인사이트 ===

1. 데이터 품질 분석:
   - 총 피처 수: 734
   - 총 샘플 수: 70560
   - 등급 분포: {'E': np.int64(56505), 'D': np.int64(10270), 'C': np.int64(3753), 'A': np.int64(28), 'B': np.int64(4)}
   - 결측치 비율: 0.022

2. 등급별 특성:
   - A등급: 28명 (0.0%)
   - B등급: 4명 (0.0%)
   - C등급: 3753명 (5.3%)
   - D등급: 10270명 (14.6%)
   - E등급: 56505명 (80.1%)

3. 모델링 권장사항:
   - 클래스 불균형 문제가 있으므로 SMOTE나 class_weight 조정 필요
   - 2단계 분류 시스템으로 E등급을 먼저 식별하는 전략 유효
   - 최근 3-6개월 데이터가 가장 중요한 예측 변수로 판단됨
   - 신용카드 사용률, 거래 빈도 등이 핵심 지표

4. 추가 개선 방안:
   - 피처 엔지니어링을 통한 비율/변화율 변수 생성
   - 시간 기반 피처 중요도를 고려한 가중치 적용
   - 앙상블 모델 적용으로 성능 향상
   - 교차 검증을 통한 모델 안정성 검증


In [13]:
# 2차 분류 시스템 구현
print(f"\n{'='*50}")
print("=== 2차 분류 시스템 구현 ===")

print("\n1단계: E등급 식별")
y_stage1 = (Y == 'E').astype(int)
print(f"E등급 비율: {y_stage1.mean():.3f}")

if y_stage1.sum() > 0:
    X1_train, X1_test, y1_train, y1_test = train_test_split(
        x_train, y_stage1, test_size=0.2, random_state=42, stratify=y_stage1
    )

    X1_balanced, y1_balanced = safe_smote_fit(X1_train, y1_train)

    models_stage1 = {
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
        'GradientBoosting': GradientBoostingClassifier(random_state=42),
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000)
    }

    best_model1 = None
    best_score1 = 0
    best_name1 = ""

    print("1단계 모델 비교 중...")
    for name, model in models_stage1.items():
        try:
            cv_scores = cross_val_score(model, X1_balanced, y1_balanced, cv=3, scoring='roc_auc')
            avg_score = cv_scores.mean()
            print(f"{name} CV AUC: {avg_score:.3f} (+/- {cv_scores.std()*2:.3f})")

            if avg_score > best_score1:
                best_score1 = avg_score
                best_model1 = model
                best_name1 = name
        except Exception as e:
            print(f"{name} 오류: {str(e)}")

    if best_model1 is not None:
        print(f"\n1단계 최적 모델: {best_name1} (AUC: {best_score1:.3f})")
        model_stage1 = best_model1
        model_stage1.fit(X1_balanced, y1_balanced)

        y1_pred = model_stage1.predict(X1_test)
        y1_pred_proba = model_stage1.predict_proba(X1_test)[:, 1]

        print(f"1단계 테스트 AUC: {roc_auc_score(y1_test, y1_pred_proba):.3f}")
        print(f"1단계 테스트 정확도: {accuracy_score(y1_test, y1_pred):.3f}")

        stage1_pred_train = model_stage1.predict(X1_train)
        non_e_predicted_indices = X1_train.index[stage1_pred_train == 0]

        actual_non_e_mask = Y != 'E'
        X_stage2 = x_train[actual_non_e_mask]
        y_stage2 = Y[actual_non_e_mask]

        if len(y_stage2.unique()) > 1 and len(y_stage2) > 10:
            print(f"2단계 데이터 크기: {X_stage2.shape}")
            print("2단계 등급 분포:")
            print(y_stage2.value_counts())

            X2_train, X2_test, y2_train, y2_test = train_test_split(
                X_stage2, y_stage2, test_size=0.2, random_state=42, stratify=y_stage2
            )

            X2_balanced, y2_balanced = safe_smote_fit(X2_train, y2_train)

            model_stage2 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
            model_stage2.fit(X2_balanced, y2_balanced)

            y2_pred = model_stage2.predict(X2_test)
            print(f"2단계 테스트 정확도: {accuracy_score(y2_test, y2_pred):.3f}")

            print("\n=== 전체 시스템 성능 ===")
            X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
                x_train, Y, test_size=0.2, random_state=42, stratify=Y
            )

            stage1_pred = model_stage1.predict(X_test_full)
            final_predictions = np.full(len(X_test_full), 'E', dtype=object)

            non_e_mask = stage1_pred == 0
            if np.any(non_e_mask):
                X_for_stage2 = X_test_full[non_e_mask]
                if len(X_for_stage2) > 0:
                    stage2_predictions = model_stage2.predict(X_for_stage2)
                    final_predictions[non_e_mask] = stage2_predictions

            overall_accuracy = accuracy_score(y_test_full, final_predictions)
            print(f"전체 시스템 정확도: {overall_accuracy:.3f}")

            print("\n등급별 분류 성능:")
            print(classification_report(y_test_full, final_predictions))

            print("\n실제 vs 예측 등급 분포:")
            print("실제:", y_test_full.value_counts().sort_index().to_dict())
            print("예측:", pd.Series(final_predictions).value_counts().sort_index().to_dict())
        else:
            print("⚠️ 2단계 분류를 위한 충분한 데이터가 없습니다.")
    else:
        print("⚠️ E등급이 존재하지 않아 2단계 분류를 수행할 수 없습니다.")
print("\n분석 완료! 모든 단계가 성공적으로 실행되었습니다.")


=== 2차 분류 시스템 구현 ===

1단계: E등급 식별
E등급 비율: 0.801
✅ SMOTE 적용 완료 (k_neighbors=5)
1단계 모델 비교 중...
RandomForest CV AUC: 0.985 (+/- 0.025)
GradientBoosting CV AUC: 0.979 (+/- 0.047)
LogisticRegression CV AUC: 0.937 (+/- 0.008)

1단계 최적 모델: RandomForest (AUC: 0.985)
1단계 테스트 AUC: 0.949
1단계 테스트 정확도: 0.904
2단계 데이터 크기: (14055, 734)
2단계 등급 분포:
Segment
D    10270
C     3753
A       28
B        4
Name: count, dtype: int64
⚠️ 일부 클래스의 샘플이 너무 적음 (최소: 3). SMOTE 적용 불가.
2단계 테스트 정확도: 0.825

=== 전체 시스템 성능 ===
전체 시스템 정확도: 0.910

등급별 분류 성능:
              precision    recall  f1-score   support

           A       1.00      0.17      0.29         6
           B       0.00      0.00      0.00         1
           C       0.74      0.47      0.58       750
           D       0.64      0.90      0.75      2054
           E       0.99      0.94      0.96     11301

    accuracy                           0.91     14112
   macro avg       0.67      0.50      0.52     14112
weighted avg       0.93      0.91      0.91 